## Evaluate the baseline correction performance of all methods and models available to us


In [ ]:
from Scripts.essentials import *
from Scripts.BaselineRemovalCopy import *
from Scripts.BubbleFillCopy import *

# Load the models

In [ ]:
ensemble_model = make_ensemble()
ensemble_model.load_weights(filepath= "Models/ensembleModelRes.h5")

standard_model = make_standard()
standard_model.load_weights(filepath = "Models/standardModel.h5")

# Cascaded model
cascade = tf.keras.models.load_model("Models/unet_three.23-64.96.h5")

In [ ]:
d_path = "data/Generated_data/"
datasets = [
           (d_path+"X_1024_10_300.npy", d_path+"y_baseline_1024_10_300.npy"),
           ]

metrics_dict = {}
full_metrics_dict = {}
for data in datasets:

    # Load the dataset we want to evaluate
    X = np.load(data[0])
    target = np.load(data[1])

    ## DL-predictions ##
    
    # Ensemble model predictions
    ensemble_preds = ensemble_model.predict(X, verbose = 0)
    # Get the baseline
    ensemble_preds = ensemble_preds[0]
    
    # Standard model
    standard_preds = standard_model.predict(X, verbose = 0)
    # Get the baseline
    standard_preds = standard_preds[0]
    
    # Cascaded preds, multiply X by 1000 to set the required input shape
    cascaded_preds = cascade.predict(X * 1000, verbose = 0)
    # Get baselines, by subtracting the baseline corrected spectrum
    cascaded_preds = (X*1000) - cascaded_preds[0]
    # Divide by 1000 to return to the normalized scope
    cascaded_preds = cascaded_preds / 1000

    # Standard methods

    # ModPoly
    mod_preds = []
    DEGREE = 5
    for x in X:
        
        baselineObj = BaselineRemoval_source(np.squeeze(x))
        pred = baselineObj.ModPoly(DEGREE) 
        # ALgorithm gives corrected spectrum, subtract from input to get baseline
        mod_preds.append(np.squeeze(x) - pred)
    
    #ImodPoly:
    imod_preds = []
    for x in X:
        
        baselineObj = BaselineRemoval_source(np.squeeze(x))
        pred = baselineObj.IModPoly(DEGREE) 
        # ALgorithm gives corrected spectrum, subtract from input to get baseline
        imod_preds.append(np.squeeze(x) - pred)
        
        
    # bubblefill:
    bubble_preds = []
    min_bubble_widths = 75
    for x in X:
        # Algorithm gives corrected spectrum and baseline itself, baseline is index 1
        pred = bubblefill(np.squeeze(x), min_bubble_widths = min_bubble_widths)[1] 
        bubble_preds.append(pred)


    method_preds = [ensemble_preds, standard_preds,
                    cascaded_preds, bubble_preds, 
                    imod_preds, mod_preds]
    
    method_names = ["Ensemble", "Standard",
                     "Kazemzadeh et al.", "Bubble fill",
                    "IModPoly", "ModPoly"]

    # Save the figures in a unique folder for potential use later
    path = "Figures/"+ data[1].split("/")[-1]
    if not os.path.exists(path):
        os.mkdir(path)
        # Gather the metrics
        
    metrics_dict[data[0]] = {}
    full_metrics_dict[data[0]] = {}
    print("####", data[0], "####")
    for preds, name in zip(method_preds, method_names):
        print(name)
        RMSE_metrics = []
        MaxError_metrics = []
        for spec, p in zip(target, np.squeeze(preds)):
            RMSE_metrics.append(root_mean_squared_error(p, spec))
            MaxError_metrics.append(metrics.max_error(spec, p))
            
        print("\tRMSE:",
              "\n\t\tMean:", np.round(np.mean(RMSE_metrics), 2),
              "\n\t\tMedian:", np.round(np.median(RMSE_metrics), 2),
              "\n\t\tVariance:", np.round(np.var(RMSE_metrics), 2))
        

    
        print("\tMax error:",
              "\n\t\tMean:", np.round(np.mean(MaxError_metrics), 2),
              "\n\t\tMedian:", np.round(np.median(MaxError_metrics), 2),
              "\n\t\tVariance:", np.round(np.var(MaxError_metrics), 2))

        metrics_dict[data[0]][name] = [np.mean(RMSE_metrics), np.median(RMSE_metrics), np.var(RMSE_metrics),
                                       np.mean(MaxError_metrics), np.median(MaxError_metrics),
                                       np.var(MaxError_metrics)]

        full_metrics_dict[data[0]][name] = [np.array(RMSE_metrics), np.array(MaxError_metrics)]

In [ ]:
path = "Data/Outputs/B/"
for data_name in full_metrics_dict.keys():
    for model_name in full_metrics_dict[data_name].keys():
        data = full_metrics_dict[data_name][model_name]

        np.save(path + model_name + ".np", data)

In [ ]:
header = ["RMSE:mean", "RMSE:median", "RMSE:variance", "MaxError:mean", "MaxError:median", "MaxError:variance"]

with pd.ExcelWriter('Results/BaselineComparisons.xlsx') as writer:
    for key in metrics_dict.keys():

        df = pd.DataFrame.from_dict(metrics_dict[key],
                                        columns = header,
                                        orient = "index")
        filename = key.split("/")[1] + key.split("/")[2]
        df = df.round(decimals = 2).drop(columns=["RMSE:median", "MaxError:median"])
        df.to_excel(writer, sheet_name=filename)
        
        print()
        df = df.style.format(decimal=',', thousands='.', precision=2)
        print(df.to_latex())

In [ ]:
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "Times New Roman"
dimension = (5, 4)
num_ticks = 4
for key in full_metrics_dict.keys():

    key2 = list(full_metrics_dict[key].keys())


    # RMSE metrics
    flier_props = dict(marker='o', markerfacecolor='gray', alpha = 0.05, markersize=5,
                  linestyle='none', markeredgecolor="gray")
    mean_props = {"marker": "_", 'markerfacecolor': "Red", 'markeredgecolor': "Red"}
    
    model_metrics = [full_metrics_dict[key][k][0] for k in key2]
    fig, ax = plt.subplots(figsize = dimension)
    ax.boxplot(model_metrics,
                  labels = key2,
                  showmeans=True, 
                  flierprops = flier_props,
                  meanprops= mean_props,
                 )

    
    ax.set_title("Root mean square error")
    max_ = np.max([full_metrics_dict[key][k][0] for k in key2])
    ticks = np.linspace(0, max_, num_ticks)
    ax.set_yticks(ticks)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)
    ax.set_xticklabels(key2, rotation = 45)
    plt.savefig("Figures/Metrics/B_"+key.split("/")[-1]+".png", format="png",
                bbox_inches='tight',
                dpi = 1000,
                transparent = True,
                pad_inches=0.5)
    plt.show()
    
    
    # Max error metrics
    model_metrics = [full_metrics_dict[key][k][1] for k in key2]
    fig, ax = plt.subplots(figsize = dimension)
    ax.boxplot(model_metrics,
                  labels = key2,
                 showmeans=True,
                  flierprops = flier_props,
                  meanprops= mean_props,
                 #showfliers=False,
                 )
    ax.set_title("Max error")
    max_ = np.max([full_metrics_dict[key][k][1] for k in key2])
    ticks = np.linspace(0, max_, num_ticks)
    ax.set_yticks(ticks)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)
    ax.set_xticklabels(key2, rotation = 45)
    plt.savefig("Figures/Metrics/B_MaxError.png", format="png",
                bbox_inches='tight',
                dpi = 1000,
                transparent = True,
                pad_inches=0.5)
    plt.show()